<a href="https://colab.research.google.com/github/Armelle-h/ML_frames_and_videos/blob/main/test_set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/ML_project_frames_and_videos')

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from functools import partial
from torch.utils.data import DataLoader
%matplotlib inline
%load_ext autoreload
%autoreload 2
! pip install ftfy #needed for CLIP to run
! pip install torchmetrics
import torchmetrics

import os
import math

from Pair_Frames import Pair_Frames

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 512 kB 4.7 MB/s 


In [ ]:
class Model_Similarity_Pair_Frames(nn.Module):
  def __init__(self):
    super().__init__()
    #Remplir les fonctions d'activations ici nn.Relu, nn.Linear
    self.input_layer = torch.nn.Linear(1024, 250) #1024, the size of 2 feature vectors concatenated, 250 the number of neurons in our hidden layers
    self.input_phi = torch.nn.ReLU()
    self.layer1 = torch.nn.Linear(250, 250) #constructed as "(number of in-edges, number of out-edges)"
    self.phi1 = torch.nn.ReLU()
    self.layer2 = torch.nn.Linear(250, 250)
    self.phi2 = torch.nn.ReLU()
    self.output_layer = torch.nn.Linear(250, 2) #our model outputs pair of values coresponding to "(P['not the same video], P['same video'])"

  def forward(self,PF):
    """PF a pair of frames"""
    #put the forward pass of the neural network using the activations functions
    PF = self.input_layer(PF)
    PF = self.input_phi(PF) #activation function on first layer
    PF = self.layer1(PF)
    PF = self.phi1(PF)
    PF = self.layer2(PF)
    PF = self.phi2(PF)
    PF = self.output_layer(PF)
    return PF

In [ ]:
#to load a trained model:

#by default loads the last model
name=os.listdir('/content/drive/MyDrive/ML_project_frames_and_videos/trained_models')[-1]
lr = 5e-3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model_Similarity_Pair_Frames()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
checkpoint = torch.load(os.path.join('/content/drive/MyDrive/ML_project_frames_and_videos/trained_models', name), map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
num_epochs = checkpoint['epoch']
criterion = checkpoint['loss']

print(f"Previously trained for {num_epochs} number of epochs and with the loss function {criterion}")

Previously trained for 2 number of epochs and with the loss function <function cross_entropy at 0x7f570ac67d30>


In [ ]:
video_dir='/content/drive/MyDrive/ML_project_frames_and_videos/DATA/videos'
length=10000 #length can be tuned
length_test=math.floor(0.15*length)
test_set = Pair_Frames(video_dir, train=2, length=length_test, device=device)

100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 125MiB/s]


In [ ]:

batch_size=128
test_loader = torch.utils.data.DataLoader(
        test_set,
        batch_size=batch_size,
    )

In [ ]:
@torch.no_grad() #'reset' gradients
def validate(model, device, test_loader, criterion):
    model.eval()  # Important set model to eval mode (affects dropout, batch norm etc)
    test_loss = 0
    correct = 0
    for batch_idx, (feature1, feature2, label) in enumerate(test_loader):
      feature1 = feature1.to(device).float() #change to device selected
      feature2 = feature2.to(device).float()
      target =label.to(device)
      if feature1.shape[0]==1: #there was an issue if the tensor had shape (1,n)
        f1=torch.flatten(feature1, 0)
        f2=torch.flatten(feature2, 0)
        data = torch.cat([f1, f2])
        output = model(data)
        output=output.reshape([1, output.shape[0]])
      else:
        data=torch.cat([feature1, feature2], dim=1)
        output = model(data)
      test_loss += criterion(output, target).item() * len(data) #weighted sum

      pred = output.argmax(dim=1, keepdim=True) #idea is that our model output a pair of proba. With this command, we keep only the index
        #indicating where the highest value is and we do this column-wise

      correct +=pred.eq(target.view_as(pred)).sum().item() #this counts the number of good answers we had when comparing to our target and adds
        #them to the number of already well predicted pair of frames.
        #the target has value 0 if "data[0]" belongs to two different videos and value 1 if it belongs to the same

    test_loss /= len(test_loader.dataset) #we renormalize as we did a weighted sum

    print(
        "Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)".format(
            test_loss,
            correct,
            len(test_loader.dataset),
            100.0 * correct / len(test_loader.dataset),
        )
    )
    return test_loss, correct / len(test_loader.dataset)

In [ ]:
validate(model, device, test_loader, criterion)

Test set: Average loss: 0.3186, Accuracy: 1318/1500 (88%)


(0.3185728220144908, 0.8786666666666667)